In [2]:
import pymongo
import datetime
import pandas as pd
import numpy as np
import cuemath_db as cuedb
import statistics

In [5]:
client_mongo = pymongo.MongoClient(cuedb.db_asli_intel_dev['host'])
inteldb = client_mongo[cuedb.db_asli_intel_dev['db_name']]
item_collection = inteldb['Items']
activity_collection = inteldb['Activities']
item_response_collection = inteldb['Item_Responses']
session_collection = inteldb['Sessions']
student_collection = inteldb['Students']
program_collection = inteldb['Programs']
chapter_collection = inteldb['Chapters']
block_collection = inteldb['Blocks']
classroom_collection = inteldb['Classrooms']

In [8]:
def get_foundation_module_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'code', 'is_assessment_live']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_module;"
    cur.execute(sql_query)
    df_module_id_code = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_module_id_code)

def get_foundation_student_attempt_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'type', 'status', 'responses', 'score_details', 'module_id', 'student_id', 'objective_responses']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_studentassessment;" # where module_id=ANY(%s);"
    cur.execute(sql_query)
    df_assessment_attempt = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_assessment_attempt)

def get_foundation_student_ma_data(segment_id):
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    cols = ['id', 'type', 'assessment_response', 'student_grade', 'skill_segment_id', 'student_id']
    sql_query = "SELECT "+", ".join(cols)+" FROM curriculum_studentskillsegmentattempt where skill_segment_id=(%s);"
    cur.execute(sql_query, [segment_id, ])
    df_ma_attempt = pd.DataFrame(cur.fetchall(), columns=cols)
    cur.close()
    conn.close()
    return(df_ma_attempt)


In [19]:
module_list = ['SM.S0.W0402.M3',\
'SM.S0.W0502.M1',\
'SM.S0.W0502.M2',\
'SM.S0.W0602.M1',\
'SM.S0.W0602.M2',\
'SM.S1.W0202.M2',\
'SM.S1.W0302.M1',\
'SM.S1.W0402.M1',\
'SM.S1.W0402.M2',\
'SM.S1.W0502.M1',\
'SM.S1.W0502.M2',\
'SM.S1.W0502.M3',\
'SM.S1.W0602.M1',\
'SM.S1.W0701.M1',\
'SM.S2.W0303.M1',\
'SM.S2.W0303.M2',\
'SM.S2.W0402.M1',\
'SM.S2.W0402.M2',\
'SM.S2.W0503.M1',\
'SM.S2.W0503.M2',\
'SM.S2.W0503.M3',\
'SM.S2.W0602.M1',\
'SM.S2.W0701.M1',\
'SM.S3.W0204.M1',\
'SM.S3.W0304.M1',\
'SM.S3.W0304.M2',\
'SM.S3.W0501.M1',\
'SM.S3.W0501.M2',\
'SM.S3.W0603.M1',\
'SM.S3.W0603.M2',\
'SM.S3.W0702.M1',\
'SM.S3.W0702.M2',\
'SM.S3.W0803.M1',\
'SM.S4.W0204.M1',\
'SM.S4.W0204.M2',\
'SM.S4.W0402.M1',\
'SM.S4.W0402.M2',\
'SM.S4.W0402.M3',\
'SM.S4.W0503.M1',\
'SM.S4.W0503.M2',\
'SM.S4.W0603.M1',\
'SM.S4.W0603.M2',\
'SM.S4.W0703.M1',\
'SM.S4.W0703.M2',\
'SM.S4.W0703.M3',\
'SM.S4.W0802.M1',\
'SM.S4.W0802.M2',\
'SM.S4.W0903.M1',\
'SM.S5.W0103.M3',\
'SM.S5.W0104.M3',\
'SM.S5.W0202.M1',\
'SM.S5.W0202.M2',\
'SM.S5.W0202.M3',\
'SM.S5.W0302.M1',\
'SM.S5.W0302.M2',\
'SM.S5.W0402.M1',\
'SM.S5.W0402.M2',\
'SM.S5.W0503.M1',\
'SM.S5.W0503.M2',\
'SM.S5.W0603.M1',\
'SM.S5.W0603.M2',\
'SM.S5.W0701.M1',\
'SM.S5.W0701.M2',\
'SM.S5.W0802.M1',\
'SM.S5.W0802.M2',\
'SM.S5.W0902.M1',\
'SM.S5.W1002.M1',\
'SM.S6.W0204.M1',\
'SM.S6.W0204.M2',\
'SM.S6.W0204.M3',\
'SM.S6.W0303.M1',\
'SM.S6.W0303.M2',\
'SM.S6.W0402.M1',\
'SM.S6.W0402.M2',\
'SM.S6.W0503.M1',\
'SM.S6.W0503.M2',\
'SM.S6.W0602.M1',\
'SM.S6.W0602.M2',\
'SM.S6.W0701.M3',\
'SM.S6.W0702.M1',\
'SM.S6.W0702.M2',\
'SM.S6.W0802.M1',\
'SM.S6.W0802.M2',\
'SM.S6.W0802.M3',\
'SM.S6.W0902.M1',\
'SM.S6.W0902.M2',\
'SM.S6.W1002.M1',\
'SM.S6.W1002.M2',\
'SM.S7.W0105.M1',\
'SM.S7.W0105.M2',\
'SM.S7.W0204.M1',\
'SM.S7.W0204.M2',\
'SM.S7.W0301.M1',\
'SM.S7.W0301.M2',\
'SM.S7.W0402.M1',\
'SM.S7.W0402.M2',\
'SM.S7.W0503.M1',\
'SM.S7.W0503.M2',\
'SM.S7.W0602.M1',\
'SM.S7.W0602.M2',\
'SM.S7.W0603.M1',\
'SM.S7.W0603.M2',\
'SM.S7.W0603.M3',\
'SM.S7.W0702.M1',\
'SM.S7.W0702.M2',\
'SM.S7.W0702.M3',\
'SM.S7.W0802.M1',\
'SM.S7.W0802.M2',\
'SM.S7.W0901.M1',\
'SM.S7.W0901.M2']
print(len(module_list))
df1 = get_foundation_module_data()
df1.to_csv('./Foundation_data/module_id_code.csv')
module_id_list = []
#for module_code in module_list:
#    module_id_list.append(df1[df1['code']==module_code].iloc[0]['id'])


110


In [9]:
df2 = get_foundation_student_attempt_data()
df2.to_csv('./Foundation_data/module_student_attempt_full.csv')

In [13]:
import json
def fetch(rows, data):
    #     print("DEBUG", data)
    a = rows["responses"]
    b=json.loads(a)
    module_id=rows["module_id"]
    student_id=rows["student_id"]
    if(data.get(module_id, None) is None):
        data.update({module_id:{}})
    for j in range(len(b)):
        if(b[j].get("code", None) is not None):
            q_code =b[j]["code"]
        elif(b[j].get("id", None) is not None):
            q_code =b[j]["id"]
        else:
            continue
        if(data[module_id].get(q_code, None) is None):
            data[module_id].update({q_code:{}})
        try:
            data[module_id][q_code].update({student_id:{"time":b[j]["time"],"result":b[j]["result"],"type":rows["type"],"status":rows["status"],"answer":b[j]["answer"]}})
        except KeyError as e:
            print("Keyerror", e)
            continue
    #             import pdb; pdb.set_trace()
    return data


def aggregate_question_data(module_id, ques_code, ques_data):
    num_attempts = 0
    num_cleared_sess_cleared = 0
    num_cleared_sess_failed = 0
    total_time = 0
    num_correct = 0
    num_incorrect = 0 
    num_A = 0
    num_B = 0 
    num_C = 0
    num_D = 0
    total_time_if_correct = 0
    total_time_if_incorrect = 0
    for student_id, dat in ques_data.items():
        num_attempts += 1
        total_time += dat['time']
        if(dat['result']):
            num_correct += 1
            total_time_if_correct += dat['time']
            if(dat['status']=='CLRD'):
                num_cleared_sess_cleared += 1
            else:
                num_cleared_sess_failed += 1
        else:
            num_incorrect += 1
            total_time_if_incorrect += dat['time']
        if(dat['answer']=="A"):
            num_A +=1
        elif(dat['answer']=="B"):
            num_B +=1
        elif(dat['answer']=="C"):
            num_C +=1
        elif(dat['answer']=="D"):
            num_D +=1
    if(num_correct!=0):
        avg_time_if_correct = total_time_if_correct/num_correct
    else:
        avg_time_if_correct = "NA"
    if(num_incorrect!=0):
        avg_time_if_incorrect = total_time_if_incorrect/num_incorrect
    else:
        avg_time_if_incorrect = "NA"
    avg_time = total_time/num_attempts
        
    # print(module_id, ques_code, num_attempts, num_correct, num_incorrect, num_cleared_sess_cleared,num_cleared_sess_failed,avg_time_if_correct,avg_time_if_incorrect, num_A,num_B,num_C,num_D,avg_time )
    out_list = [module_id, ques_code, num_attempts, num_correct, num_incorrect, num_cleared_sess_cleared,num_cleared_sess_failed,avg_time_if_correct,avg_time_if_incorrect, num_A,num_B,num_C,num_D,avg_time]
    out_final_list = [str(a) for a in out_list]
    with open(fl_nm, "a") as fp:
        fp.write(", ".join(out_final_list) + '\n')

In [14]:
ques_dict = {}
# unique_modules = df_module_attempt['module_id'].unique()
# print(unique_students)
#module_attempt_subset = df_module_attempt[df_module_attempt['module_id']==unique_modules[1]]
df_module_attempt = df2
print(df_module_attempt.shape[0])
for i in range(df_module_attempt.shape[0]):
    ques_dict = fetch(df_module_attempt.iloc[i], ques_dict)
    # ques_dict.update(attempt_row)
print("DONE", len(ques_dict))


761656
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) b
(Pdb) b[j]
*** The specified object '[j]' is not a function or was not found along sys.path.
(Pdb) b
(Pdb) a


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) b
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
Keyerror 'result'
> <ipython-input-13-c9ca017ccbd8>(10)fetch()
-> for j in range(len(b)):
(Pdb) c
DONE

In [17]:
with open('./Foundation_data/module_ques_data.json', 'w') as outfile:
    json.dump(ques_dict, outfile)

In [22]:
fl_nm = './Foundation_data/main_out_modules_final_all.csv'
df_module_id = df1
with open(fl_nm, "w") as fp:
    fp.write('Module_id, Question Code, Num Attempts,Num Correct, Num Incorrect, Num Correct Cleared Session,Num Correct failed session,Time if Correct,Time if Incorrect, Num A,Num B,Num C,Num D,Average Time \n')
for module_id, module_obj in ques_dict.items():
    for q_code, ques_obj in module_obj.items():
        module_code = df_module_id[df_module_id['id']==module_id].iloc[0]['code']
        aggregate_question_data(module_code, q_code, ques_obj)
    

SM.S2.W2.M3 G2W2M3_12 380 188 192 113 75 33024.882978723406 24302.557291666668 44 24 124 188 28617.813157894736
SM.S2.W2.M3 G2W2M3_24 351 211 140 122 89 22216.317535545022 16959.621428571427 24 38 78 211 20119.62962962963
SM.S2.W2.M3 G2W2M3_21 361 221 140 121 100 43976.69683257918 33403.44285714286 27 39 74 221 39876.26592797784
SM.S2.W2.M3 G2W2M3_3 333 162 171 87 75 38717.83950617284 25833.175438596492 31 67 162 73 32101.390390390392
SM.S2.W2.M3 G2W2M3_16 363 229 134 135 94 50131.64628820961 31282.768656716416 40 30 64 229 43173.65840220386
SM.S2.W2.M3 G2W2M3_29 475 253 222 210 43 28908.517786561264 17618.73873873874 253 94 60 68 23632.031578947368
SM.S2.W2.M3 G2W2M3_17 476 260 216 221 39 24009.784615384615 19675.40740740741 260 50 127 39 22042.9243697479
SM.S2.W2.M3 G2W2M3_4 461 239 222 197 42 28379.359832635982 17631.364864864863 95 90 37 239 23203.53579175705
SM.S2.W2.M3 G2W2M3_30 465 307 158 242 65 26325.130293159607 11494.753164556962 49 307 63 46 21285.991397849462
SM.S2.W2.M3 G

SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_34 543 423 120 297 126 11492.25059101655 17723.808333333334 0 0 0 0 12869.390423572744
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_22 518 357 161 264 93 21808.526610644258 25712.329192546586 0 0 0 0 23021.870656370655
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_26 541 485 56 339 146 13139.901030927835 13590.982142857143 7 16 33 485 13186.593345656192
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_36 568 338 230 338 0 22897.21301775148 16592.291304347826 147 26 57 338 20344.163732394365
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_35 567 395 172 395 0 23943.607594936708 20540.61046511628 395 69 62 41 22911.30511463845
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_3 473 303 170 169 134 11255.712871287129 10894.123529411765 12 92 303 66 11125.754756871036
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_5 468 241 227 136 105 32427.55601659751 23176.79295154185 12 68 147 241 27940.540598290598
SM.S5.X2.M1 T1L6M4-Grade5_JAtHA1t_15 479 283 196 187 96 17543.73851590106 15504.352040816326 0 0 0 0 16709.25052192067
SM.S5.X2.M1 T1L

SM.S5.W0201.M1 G5W2M1_N7lwsN7_3 1270 1051 219 417 634 15328.184586108468 14327.10502283105 44 61 114 1051 15155.557480314961
SM.S5.W0201.M1 G5W2M1_N7lwsN7_13 1314 802 512 383 419 42408.99002493766 26456.498046875 137 253 122 802 36193.102739726026
SM.S5.W0201.M1 G5W2M1_N7lwsN7_18 1242 492 750 298 194 61075.08943089431 39783.16533333333 0 0 0 0 48217.647342995166
SM.S5.W0201.M1 G5W2M1_N7lwsN7_22 1298 875 423 426 449 11096.828571428572 8812.07328605201 306 875 47 70 10352.258859784284
SM.S5.W0201.M1 G5W2M1_N7lwsN7_30 1260 539 721 327 212 58305.38961038961 27681.066574202498 130 135 539 456 40781.47142857143
SM.S5.W0201.M1 G5W2M1_N7lwsN7_33 1177 498 679 303 195 53261.170682730924 26163.751104565537 215 498 247 217 37628.92948173322
SM.S5.W0201.M1 G5W2M1_N7lwsN7_4 1350 1202 148 486 716 12125.17387687188 11559.52027027027 41 1202 64 43 12063.161481481482
SM.S5.W0201.M1 G5W2M1_N7lwsN7_7 1350 882 468 394 488 23879.52267573696 20149.273504273504 151 882 138 179 22586.36962962963
SM.S5.W0201.M1

SM.S4.W7.M1 G4W7M1_Grade4_28 516 155 361 132 23 29330.832258064514 19842.806094182826 220 58 83 155 22692.891472868218
SM.S4.W7.M1 G4W7M1_Grade4_30 530 411 119 344 67 15418.131386861314 13273.974789915967 44 411 50 25 14936.707547169812
SM.S4.W7.M1 G4W7M1_Grade4_24 537 289 248 249 40 24508.58131487889 17299.907258064515 67 65 116 289 21179.435754189944
SM.S4.W7.M1 G4W7M1_Grade4_10 469 352 117 228 124 9527.011363636364 10021.384615384615 61 352 42 14 9650.341151385928
SM.S4.W7.M1 G4W7M1_Grade4_15 610 521 89 393 128 7375.332053742803 7994.224719101124 49 17 521 23 7465.629508196721
SM.S4.W7.M1 G4W7M1_Grade4_31 567 433 134 346 87 15221.965357967667 14469.746268656716 65 35 34 433 15044.192239858907
SM.S4.W7.M1 G4W7M1_Grade4_22 570 433 137 338 95 14251.415704387991 13586.890510948906 57 433 54 26 14091.696491228071
SM.S4.W7.M1 G4W7M1_Grade4_8 506 401 105 326 75 20913.693266832917 13655.704761904763 32 29 44 401 19407.588932806324
SM.S4.W7.M1 G4W7M1_Grade4_17 505 308 197 249 59 15683.668831

SM.S3.W2.M1 G3W2M1_xzp8T42_44 372 252 120 160 92 12267.047619047618 11313.833333333334 252 22 80 18 11959.559139784946
SM.S3.W2.M1 G3W2M1_xzp8T42_42 372 239 133 147 92 41984.86192468619 26734.38345864662 61 37 239 35 36532.405913978495
SM.S3.W2.M1 G3W2M1_xzp8T42_10 460 222 238 181 41 22341.46846846847 17091.7268907563 193 222 34 11 19625.297826086957
SM.S3.W2.M1 G3W2M1_xzp8T42_17 443 229 214 171 58 39816.34061135371 25709.630841121496 229 123 37 54 33001.812641083525
SM.S3.W2.M1 G3W2M1_xzp8T42_38 441 216 225 159 57 44051.56018518518 21653.346666666668 216 57 109 59 32623.90022675737
SM.S3.W2.M1 G3W2M1_xzp8T42_47 445 274 171 205 69 31476.054744525547 17522.204678362574 54 72 45 274 26114.013483146067
SM.S3.W2.M1 G3W2M1_xzp8T42_31 347 185 162 126 59 26967.891891891893 22380.845679012345 93 38 185 31 24826.389048991354
SM.S3.W2.M1 G3W2M1_xzp8T42_40 469 376 93 293 83 24215.98404255319 14275.849462365592 24 36 33 376 22244.912579957356
SM.S3.W2.M1 G3W2M1_xzp8T42_5 454 364 90 255 109 16516.0

...

SM.S4.D0503.M2 T2L2M5_G4W5M2_Grade4_15 1 1 0 1 0 20784.0 NA 0 0 1 0 20784.0
SM.S4.D0503.M2 T2L2M5_G4W5M2_Grade4_23 1 0 1 0 0 NA 24090.0 0 0 0 1 24090.0
SM.S4.D0503.M2 T2L2M5_G4W5M2_Grade4_32 1 0 1 0 0 NA 23842.0 1 0 0 0 23842.0
SM.S4.D0503.M2 G4W5M2_2_49 1 1 0 1 0 33517.0 NA 0 0 0 1 33517.0
SM.S4.D0503.M2 T2L2M5_G4W5M2_Grade4_34 1 0 1 0 0 NA 8738.0 0 0 1 0 8738.0
SM.S4.D0802.M1 G4W8M1_3 1 1 0 1 0 30194.0 NA 0 1 0 0 30194.0
SM.S4.D0802.M1 G4W8M1_4 1 0 1 0 0 NA 45573.0 1 0 0 0 45573.0
SM.S4.D0802.M1 G4W8M1_8 1 1 0 1 0 17498.0 NA 0 1 0 0 17498.0
SM.S4.D0802.M1 G4W8M1_10 1 1 0 1 0 38675.0 NA 0 0 1 0 38675.0
SM.S4.D0802.M1 G4W8M1_15 1 1 0 1 0 21074.0 NA 0 0 0 1 21074.0
SM.S4.D0802.M1 G4W8M1_17 1 0 1 0 0 NA 32482.0 0 1 0 0 32482.0
SM.S4.D0802.M1 G4W8M1_20 1 1 0 1 0 12185.0 NA 0 1 0 0 12185.0
SM.S4.D0802.M1 G4W8M1_23 1 0 1 0 0 NA 62361.0 0 0 1 0 62361.0
SM.S4.D0802.M1 G4W8M1_27 1 1 0 1 0 42995.0 NA 1 0 0 0 42995.0
SM.S4.D0802.M1 G4W8M1_29 1 1 0 1 0 14116.0 NA 1 0 0 0 14116.0
SM.S4.D0802.M1 G4

SM.S6.X7.M1 T7L1M1-Grade7_35 26 2 24 2 0 39882.0 31884.083333333332 0 0 0 0 32499.30769230769
SM.S6.X7.M1 T7L1M1-Grade7_31 25 18 7 12 6 14497.5 8559.0 18 3 3 1 12834.72
SM.S6.X7.M1 T7L1M1-Grade7_15 28 23 5 16 7 18463.260869565216 16245.6 23 0 2 3 18067.25
SM.S6.X7.M1 T7L1M1-Grade7_3 23 4 19 3 1 7944.75 10252.105263157895 16 2 1 4 9850.826086956522
SM.S6.X7.M1 T7L1M1-Grade7_1 19 16 3 10 6 16379.875 13777.333333333334 1 0 16 2 15968.947368421053
SM.S6.X7.M1 T7L1M1-Grade7_21 25 11 14 10 1 23771.81818181818 13052.42857142857 3 6 5 11 17768.96
SM.S6.X7.M1 T7L1M1-Grade7_24 22 0 22 0 0 NA 32151.0 0 0 0 0 32151.0
SM.S6.X7.M1 T7L1M1-Grade7_11 30 22 8 15 7 13998.954545454546 11483.875 5 3 22 0 13328.266666666666
SM.S6.X7.M1 T7L1M1-Grade7_16 26 11 15 7 4 13332.636363636364 12128.133333333333 9 6 11 0 12637.73076923077
SM.S6.X7.M1 T7L1M1-Grade7_9 24 13 11 11 2 33322.46153846154 12546.545454545454 4 2 5 13 23800.166666666668
SM.S6.X7.M1 T7L1M1-Grade7_22 21 11 10 7 4 23003.0 8891.2 5 11 3 2 16283.09

SM.S6.X7.M2 T7L1M2_G6W6M2_Grade6_2 12 1 11 0 1 17327.0 12325.90909090909 2 1 4 5 12742.666666666666
SM.S6.X7.M2 T7L1M2_G6W6M2_Grade6_11 14 12 2 4 8 10751.916666666666 33780.0 2 0 12 0 14041.642857142857
SM.S6.X4.M3 T2L3M3_Grade5_15 52 47 5 22 25 10415.574468085106 6670.2 2 3 47 0 10055.442307692309
SM.S6.X4.M3 T2L3M3_Grade5_4 48 35 13 22 13 17952.82857142857 12724.692307692309 5 7 1 35 16536.875
SM.S6.X4.M3 T2L3M3_Grade5_16 46 34 12 16 18 26289.5 11108.833333333334 5 4 3 34 22329.32608695652
SM.S6.X4.M3 T2L3M3_Grade5_2 45 33 12 15 18 21273.484848484848 13647.25 6 33 1 5 19239.82222222222
SM.S6.X4.M3 T2L3M3_Grade5_22 49 44 5 27 17 9405.772727272728 6741.4 4 44 1 0 9133.897959183674
SM.S6.X4.M3 T2L3M3_Grade5_8 47 33 14 26 7 21167.636363636364 7501.428571428572 6 8 0 33 17096.85106382979
SM.S6.X4.M3 T2L3M3_Grade5_6 50 37 13 26 11 19800.45945945946 17585.846153846152 8 37 4 1 19224.66
SM.S6.X4.M3 T2L3M3_Grade5_21 55 53 2 32 21 7553.018867924528 2261.0 53 2 0 0 7360.581818181819
SM.S6.X4.M3

SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_DJZfqcm_34 46 27 19 19 8 28754.48148148148 17997.42105263158 2 27 10 7 24311.347826086956
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_DJZfqcm_5 43 24 19 18 6 15318.791666666666 5157.8421052631575 24 11 6 2 10829.06976744186
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_DJZfqcm_33 40 20 20 18 2 37419.6 16692.8 20 8 6 6 27056.2
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_24 25 22 3 11 11 11289.545454545454 11827.333333333334 2 1 0 22 11354.08
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_2 17 10 7 4 6 11525.9 11853.285714285714 1 10 0 6 11660.70588235294
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_11 20 15 5 5 10 13803.133333333333 15132.6 4 0 15 1 14135.5
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_19 25 20 5 7 13 8333.25 7064.6 0 4 20 1 8079.52
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_7 27 14 13 6 8 22319.071428571428 10436.846153846154 3 7 14 3 16598.0
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_38 22 3 19 1 2 4967.333333333333 13447.78947368421 3 3 7 9 12291.363636363636
SM.S7.X5.M2 T7L1M2_G6W6M2_Grade6_32 22 9 13 5 4 22017.11111111111 60

SM.S5.D0603.M2 G5W6M2_Grade5_5 1 1 0 1 0 31512.0 NA 1 0 0 0 31512.0
SM.S5.D0603.M2 G5W6M2_Grade5_24 1 1 0 1 0 49323.0 NA 0 0 0 1 49323.0
SM.S5.D0603.M2 G5W6M2_Grade5_13 1 1 0 1 0 39832.0 NA 1 0 0 0 39832.0
SM.S5.D0603.M2 G5W6M2_Grade5_20 1 1 0 1 0 65631.0 NA 0 0 0 1 65631.0
SM.S5.D0603.M2 G5W6M2_Grade5_15 1 1 0 1 0 60720.0 NA 0 0 1 0 60720.0
SM.S5.D0603.M2 G5W6M2_Grade5_26 1 1 0 1 0 39485.0 NA 0 1 0 0 39485.0
SM.S5.D0603.M2 G5W6M2_2_37 1 0 1 0 0 NA 71748.0 0 1 0 0 71748.0
SM.S5.D0603.M2 G5W6M2_Grade5_28 1 1 0 1 0 39597.0 NA 0 0 0 1 39597.0
SM.S5.D0603.M2 G5W6M2_2_41 1 1 0 1 0 150661.0 NA 0 0 0 0 150661.0


In [5]:
df3 = get_foundation_student_ma_data('429c3d8a-302c-11e7-9151-025cb0956c9b')
df3.to_csv('./Foundation_data/module_student_ma_full.csv')

In [20]:
import json
a = df2.iloc[0]['responses']
b = json.loads(a)
# print(b[0])
if(b[0].get('code', None) is not None):
    code = b[0]['code']
elif(b[0].get('id', None) is not None):
     code = b[0]['id']
else:
    raise KeyError

{'code': 'G2W2M3_12', 'time': 58564, 'answer': 'D', 'result': True, 'answer_index': 0}


In [2]:
item_score_dict = {}
item_cuescore_dict = {}
item_time_dict = {}
num_widgets = []
for item_response in item_response_collection.collection.find({}):
    #print(item_response)
    num_widgets.append(len(item_response['responses']))
    try:
        
        item_time_dict[item_response['source']['reference']].append(item_response['time'])
        item_score_dict[item_response['source']['reference']].append(float(item_response['score'])/item_response['max_score'])
        item_cuescore_dict[item_response['source']['reference']].append(float(item_response['cuescore'])/item_response['max_score'])
    except KeyError:
        item_time_dict[item_response['source']['reference']]=[item_response['time']]
        item_score_dict[item_response['source']['reference']] = [(float(item_response['score'])/item_response['max_score'])]    
        item_cuescore_dict[item_response['source']['reference']]=[float(item_response['cuescore'])/item_response['max_score']]


In [14]:
from update_db_script import *
df_student_classroom_ratings = get_classroom_ratings_data(default_date)
for idx, row in df_student_classroom_ratings.iterrows():
    ratings_obj = row.to_dict()
    ratings_obj['rating'] = float(ratings_obj['rating'])
    ratings_obj['created_on'] = str(ratings_obj['created_on'])
    ratings_obj['id'] = str(ratings_obj['id'])
    print(ratings_obj)
    classroom_collection.collection.update_one({'_id':row['student_classroom_id']}, {'$set':{'ratings':ratings_obj}})
    


{'id': '653f28f4-20ba-11e9-9446-02420a000162', 'meta': {'feedback': 'Test comment'}, 'created_on': '2019-01-25 21:30:54.615499', 'rating': 5.0, 'student_classroom_id': '967c4d9e-20b9-11e9-a10b-02420a000172'}
{'id': 'c87f196c-20bd-11e9-b5fd-02420a000156', 'meta': {'feedback': 'test'}, 'created_on': '2019-01-25 21:55:09.619159', 'rating': 4.0, 'student_classroom_id': '69c5d6f0-20bc-11e9-a10b-02420a000172'}
{'id': '613852b2-20c4-11e9-8714-02420a000149', 'meta': {'feedback': 'test3\n'}, 'created_on': '2019-01-25 22:42:22.827119', 'rating': 3.0, 'student_classroom_id': 'a17c3452-20c3-11e9-a2af-02420a000145'}
{'id': '9d2f82b2-2197-11e9-a957-02420a000147', 'meta': {}, 'created_on': '2019-01-26 23:54:27.242600', 'rating': 4.0, 'student_classroom_id': 'a7f858a8-20d0-11e9-846b-02420a000145'}
{'id': '3e3f9444-2260-11e9-a2af-02420a000145', 'meta': {'feedback': '-'}, 'created_on': '2019-01-27 23:50:36.806995', 'rating': 3.0, 'student_classroom_id': 'd262fee4-2199-11e9-846b-02420a000145'}
{'id': 'ae

In [5]:
students = student_collection.collection.find()
with open("./tmp/students_attendance.csv", "w") as fp:
    fp.write("Name, Attendance, Max Attendance \n")
for student in students:
    classrooms = classroom_collection.collection.find({'student_id':student['_id']})
    attendance, attendance_max = 0, 0
    for classroom in classrooms:
        if(classroom['status'] == "P"):
            attendance += 1
        if(classroom['status'] != "TA"):
            attendance_max += 1
    name = student['first_name'] + " " + student['last_name']
    with open("./tmp/students_attendance.csv", "a") as fp:
        fp.write(", ".join([name, str(attendance), str(attendance_max)]) + '\n')

In [75]:
fl_nm = "./Analysis/scores_time_data_mean.csv"
with open(fl_nm, "w") as fp:
    fp.write("Item_Reference, mean_score, mean_time, mean_cuescore \n")
for k in item_score_dict.keys():
    #score_sav = {k:statistics.median(v)}
    v1 = item_score_dict[k]
    v2 = item_time_dict[k]
    v3 = item_cuescore_dict[k]
    with open(fl_nm, "a") as fp:
        fp.write((k +"," + str(statistics.mean(v1)) +","+ str(statistics.mean(v2)) +","+str(statistics.mean(v3)) +"\n")) 

In [69]:
trig_2_threshold = 7*60
trig_3_threshold = 10*60
trig_5_threshold = 0.5
fl_nm = "./Analysis/ram_streak_length.csv"
with open(fl_nm, "w") as fp:
    fp.write("Session_id, User_id, Mean_Streak_length, Max_streak_length \n")
for session in session_collection.collection.find({}):
    #trigger_1, trigger_2, trigger_3, trigger_4, trigger_5 = False, False, False, False, False
    #tm_queue = [0, 0, 0]
    #score_correctness_queue = [True, True, True, True, True]
    #score_queue = [0, 0, 0]
    incorrect_streaks = []
    #max_streak = 0
    streak = 0
    for idx, item_resp_id in enumerate(session['item_responses']):
        item_response = item_response_collection.collection.find_one({'_id': item_resp_id})
        if(item_response['score'] != item_response['max_score']):
            streak += 1
        else:
            incorrect_streaks.append(streak)
            streak = 0
        '''
        if(item_response['time']>trig_2_threshold):
            trigger_2 = True
        #print((idx%3))
        #print(tm_queue[0])
        tm_queue[int(idx%3)] = item_response['time']
        if(sum(tm_queue) > trig_3_threshold):
            trigger_3 = True
        score_correctness_queue[int(idx%5)] = (item_response['score'] == item_response['max_score'])
        score_queue[int(idx%3)] = float(item_response['score'])/item_response['max_score']
        if(sum(score_queue) < trig_5_threshold):
            trigger_5 = True
        if(not any(score_correctness_queue)):
            trigger_4 = True
        '''
    max_streak = max(incorrect_streaks)
    if(len(incorrect_streaks)!=0):
        avg_incorrect_streak = sum(incorrect_streaks)/len(incorrect_streaks)
    else:
        avg_incorrect_streak = 0
    with open(fl_nm, "a") as fp:
        fp.write(", ".join([session['_id'], session['user_id'], str(avg_incorrect_streak), str(max_streak)])+"\n")
    

In [55]:
student_list = ['09300b30-d5f8-11e8-9bfb-02420a000032',
'101be654-d764-11e8-9615-02420a0000ad',
'156dfff4-fde6-11e8-ab68-02420a0000da',
'1592106c-f6dc-11e8-b02f-02420a0000e9',
'16e709fc-cbaa-11e8-be07-02420a00002d',
'17354f72-cbaa-11e8-be07-02420a00002d',
'1c77b780-df60-11e8-90cd-02420a0000a0',
'2098bb60-ed99-11e8-8561-02420a00000b',
'24837f56-043e-11e9-ab2f-02420a0000d9',
'2977f8b6-d12b-11e8-9aae-02420a00002b',
'2a1277a4-f2f2-11e8-b02f-02420a0000e9',
'2aa287ac-01e5-11e9-862f-02420a0000d8',
'2b7d0860-0766-11e9-a7fb-02420a0000ba',
'2e122fe6-d5e4-11e8-90d6-02420a000030',
'2f957168-02b5-11e9-aa77-02420a0000d3',
'3100de78-f876-11e8-ba85-02420a0000cc',
'31523f82-01dd-11e9-aa77-02420a0000d3',
'360759a6-db5c-11e8-b7b8-02420a0000b1',
'37185d70-fa03-11e8-ab01-02420a0000d5',
'3787e2ae-0371-11e9-b7c0-02420a0000dc',
'43121716-d5f9-11e8-a0a7-02420a00002f',
'44ffb53a-0513-11e9-862f-02420a0000d8',
'46457a72-0f15-11e9-82fb-02420a00009f',
'46fb3052-e336-11e8-a044-02420a0000a5',
'4900dcce-fa08-11e8-b891-02420a0000d7',
'4a89fed2-043e-11e9-ab68-02420a0000da',
'56b5f4be-f6dc-11e8-b02f-02420a0000e9',
'585e9c8a-020d-11e9-b46b-02420a0000dd',
'5d05c808-f934-11e8-b7c0-02420a0000dc',
'6488b9dc-d5f8-11e8-a061-02420a00002c',
'664f6258-cacb-11e8-ac43-02420a00002d',
'6b3a9c5a-d1e8-11e8-a061-02420a00002c',
'6de667dc-0a7c-11e9-b344-02420a0000a4',
'72b8567a-f876-11e8-9c65-02420a0000c5',
'7304b9a8-fc58-11e8-ab01-02420a0000d5',
'7441e526-fa04-11e8-9b03-02420a0000d3',
'7ac095fe-ff87-11e8-b891-02420a0000d7',
'7b5724b6-0742-11e9-b8b3-02420a0000b4',
'7faae4a8-db73-11e8-8fdf-02420a0000ac',
'8030e254-f6dc-11e8-ab87-02420a00001a',
'88480c3c-0902-11e9-a7fb-02420a0000ba',
'899db63e-e96a-11e8-b51b-02420a00004e',
'8d7e1768-d1ea-11e8-b4c6-02420a000034',
'930a3954-02b5-11e9-862f-02420a0000d8',
'a002b8aa-dcfb-11e8-84fc-02420a0000b2',
'a236918a-ed9c-11e8-9e03-02420a0000d8',
'a4d01514-fd29-11e8-ab2f-02420a0000d9',
'a51a0e06-d5f8-11e8-a0a7-02420a00002f',
'a97aa3e4-f875-11e8-ad66-02420a0000cb',
'ab3ab58a-e99c-11e8-9ee2-02420a000054',
'b01dc678-f3ce-11e8-9b4a-02420a0000e2',
'ba26fd52-c501-11e8-a7d2-02420a0000dd',
'ba759b10-c501-11e8-a7d2-02420a0000dd',
'bac39e14-c501-11e8-a7d2-02420a0000dd',
'bb0dd27c-c501-11e8-a7d2-02420a0000dd',
'bb549dbe-cad9-11e8-8a2f-02420a00002d',
'be42265e-ed98-11e8-92b1-02420a0000f6',
'c3daa580-f934-11e8-a372-02420a0000da',
'c81f2c4a-d762-11e8-84fc-02420a0000b2',
'ca3f5216-f161-11e8-b723-02420a000021',
'cb35e394-d5f8-11e8-b4c6-02420a000034',
'ccd5582e-d12a-11e8-be42-02420a00002d',
'cd91e128-d1e9-11e8-bc88-02420a00002e',
'ce694602-f3ba-11e8-b723-02420a000021',
'd0a7b76e-cbb0-11e8-ac8a-02420a00002d',
'd1013b22-cbb0-11e8-ac8a-02420a00002d',
'd2c71e08-d06b-11e8-8780-02420a00002a',
'd840754c-d1eb-11e8-ad72-02420a000035',
'e33d4974-d822-11e8-b857-02420a0000b7',
'ec0b62f4-f305-11e8-8561-02420a00000b',
'ed37f10e-cba2-11e8-8bfe-02420a00002d',
'f023a952-09b5-11e9-b29b-02420a000077',
'f51f37cc-02c2-11e9-baf6-02420a0000de',
'f706271e-eef2-11e8-ab7f-02420a00001d',
'f9e9a944-ebda-11e8-8688-02420a0000ae',
'fc491b36-f6e1-11e8-a967-02420a000012',
'fc7eb524-e3f5-11e8-83fe-02420a0000c7',
'ff2388fa-fd3a-11e8-9062-02420a0000de',
'ff75a84e-ed98-11e8-9e03-02420a0000d8']

In [56]:
my_dict = student_collection.collection.find({'is_teacher':False})
my_list = []
for obj in my_dict:
    my_list.append(obj['_id'])


In [28]:
from update_from_cuedb import *

df1, df2, df3  = get_av_data_by_date('2019-01-14')#, 'cd91e128-d1e9-11e8-bc88-02420a00002e')#, {'item_response_dict': {'38fb69f5-26a7-4c41-b92d-052fb534efe6': {'7a119db3-04b4-4ba0-a8a5-b34b08eadb4e': '2018-10-20T13:54:34Z', 'cb6bb3b7-dffe-4470-9160-783177bed4f6': '2018-10-20T13:54:54Z', 'eb7b0b03-3383-4d66-998c-cd49a24fd31e': '2018-10-20T13:54:58Z', 'c0e919b5-040e-465f-9513-65fea5a438bb': '2018-10-20T13:55:24Z', '2938f4a3-5908-43fb-b418-bcd119de0d7b': '2018-10-20T13:55:44Z', 'a70fb0e9-be3f-43e4-9344-096b6453fb79': '2018-10-20T13:56:00Z', '8f4d85a3-7b1c-4b6f-8e09-bc668178001b': '2018-10-20T13:56:18Z'}, '1ae9be47-1748-4c39-979d-e88ee2919855': {'d3924d9c-e8a5-4d08-bc32-da5181b1d67e': '2018-10-20T13:56:39Z'}, '30a9b8f9-b3a6-41c7-bf45-0229e00e7c5a': {'37b2696c-abc6-406c-8fbf-03b4e9ec51cd': '2018-10-20T13:57:16Z'}, '4df78e01-0d06-4e24-8dc6-7dcd2f095fb6': {'f66d3160-49a7-4608-8f02-4147eb33e154': '2018-10-20T13:58:11Z'}, 'c396b76e-2172-443f-848c-632485ee69be': {'757be90c-e6e0-49a3-9f2d-fd9335cd1211': '2018-10-20T13:59:08Z'}, '61aab033-e3ee-406d-b677-a4382a8c9662': {'843595a4-25d9-4faa-880a-2cc2c91dbb6d': '2018-10-20T14:01:50Z'}, '689cea4a-82d9-43ea-a410-b37511b8ffb9': {'c26157f1-661d-4f90-9235-359d3b0452af': '2018-10-20T14:02:12Z', '5f3fc8cd-5091-447e-8413-66c730858fc8': '2018-10-20T14:02:35Z', 'cacfc4a9-f4b1-4747-99df-6ec492c83b6a': '2018-10-20T14:03:35Z'}, '31670515-22a7-4c73-b751-4c46bf9a9269': {'82a2a85c-9c10-4e48-8a87-229b909aa442': '2018-10-20T14:03:53Z', 'd952e977-57a7-44f2-83e9-6c1aea9ba50b': '2018-10-20T14:04:06Z', '16d7b023-c1f8-4bcf-81d6-1b8b394d44db': '2018-10-20T14:04:22Z'}, 'f273df08-6511-4ccf-817b-a4944a4e03a1': {'2db6fe47-6113-467c-9a8b-582a758c0d1a': '2018-10-20T14:04:39Z', '1f238094-45a4-48db-81ed-ab55b785bfb1': '2018-10-20T14:05:09Z'}, '22751759-0514-4f7a-a761-cefbec88fa9a': {'d79daf0e-6711-4bb4-a4e1-f13586e2b0e4': '2018-10-20T14:05:53Z', 'a2c556dd-0c4f-4ca6-a62b-c201812c7797': '2018-10-20T14:09:06Z'}, '231723bb-f95b-4d02-a414-e439be281e1b': {'bf0849f1-f161-4cd8-8957-e61f42d0bb12': '2018-10-20T14:09:25Z', 'd0dd61f8-e388-4618-8550-02102d3f7aaf': '2018-10-20T14:11:40Z'}, '7cfde870-884b-4f6a-b8b3-51121d809318': {'9be46987-2e1b-44ba-b4bd-21a18843ec3e': '2018-10-20T14:12:18Z'}, 'be340c44-28ad-4f17-a6be-569bffd3ac7c': {'4d17e5c5-9ead-4313-8b7d-9938d70d6b47': '2018-10-20T14:12:51Z'}, '09c18a70-7a73-454e-a626-651d1d0a173c': {'76b153f3-ef55-429c-8f0f-fcb626f51817': '2018-10-20T14:13:32Z'}}, 'total_time': 1085, 'total_score': 43.0, 'total_cuescore': 38.5, 'total_max_score': 47.0, 'total_num_attempts': 31, 'total_num_questions': 27, 'sheet_type': 'STUDENT_LESSON', 'sheet_completed': True})
#df1.drop_duplicates(subset=['video_log_url', "stream_id"], )
#df1.shape[0]
#df1[(df1['id']=='2fccf166-d6d1-11e8-aff7-02420a00000e')]
df2[df2['requested_by']=='4900dcce-fa08-11e8-b891-02420a0000d7']


,id,created_on,updated_on,meta,requested_by,requested_on,is_handled


In [17]:
#sessions = {'1_MX40NjE4NzU5Mn5-MTU0MDMwMDU1MDkxNH4wV0dxaGR3VFJ3Szl6b0NLTWxDVTl6R0x-QX4', '1_MX40NjE4NzU5Mn5-MTU0MDI5NTk0OTA2N35wcWZiOS9TT3B1Qi9Cc05pclBud2hKaFF-QX4', '2_MX40NjE4NzU5Mn5-MTU0MDMwMDU3OTU0Mn5ISFFacktNdUVwc21PK1U4U2FKTXF2Uzh-QX4', '1_MX40NjE4NzU5Mn5-MTU0MDMwMDk1NjI0NH5ycWNnMXY2Rk9QZFMwMnVIdTBkWXdhV1l-QX4', '1_MX40NjE4NzU5Mn5-MTU0MDMwMDc1Njk0N340VlNhQTFkZHJCUjBZcVdqa25nR2tSejJ-QX4'}
sessions = df1[(df1["user_id"]=='d840754c-d1eb-11e8-ad72-02420a000035') & (df1["user_role"]=="publisher")]['session_id'] 
set(sessions)

{'1_MX40NjE4NzU5Mn5-MTU0MDgxNjA2ODQ0Nn4yaDZrRHNGMVZUZzhoRWRWZzdXUms3bVN-QX4',
 '1_MX40NjE4NzU5Mn5-MTU0MDgxODkxMDM0Mn5QT1c4U0NkRVpoVGNYU1RLVmorN0lRdHh-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxODk3MzQ3NX5vWmRLb0U5TjNxS1EyaDlROG5pekFmc2R-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxODkxNTE4MX5lalphM3ZxODdjUmpyVitMT1ZWTUY1elp-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxOTEyNzk4NH5DNll5OTJWQzF6NXM1T20vSDV5enZXZzV-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgyMDIxMDM3N35BOUMydUxMUVRtKy9XbzU5WkNLb0cyY1R-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgyMDMxNDA4NH53SFFTTVNBSW9sTU5zeCsvNGhIS3h3ak5-QX4'}

In [1]:
from update_from_cuedb import *
session_ids = ['1_MX40NjE4NzU5Mn5-MTU0MDgxNjA2ODQ0Nn4yaDZrRHNGMVZUZzhoRWRWZzdXUms3bVN-QX4',
 '1_MX40NjE4NzU5Mn5-MTU0MDgxODkxMDM0Mn5QT1c4U0NkRVpoVGNYU1RLVmorN0lRdHh-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxODk3MzQ3NX5vWmRLb0U5TjNxS1EyaDlROG5pekFmc2R-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxODkxNTE4MX5lalphM3ZxODdjUmpyVitMT1ZWTUY1elp-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgxOTEyNzk4NH5DNll5OTJWQzF6NXM1T20vSDV5enZXZzV-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgyMDIxMDM3N35BOUMydUxMUVRtKy9XbzU5WkNLb0cyY1R-QX4',
 '2_MX40NjE4NzU5Mn5-MTU0MDgyMDMxNDA4NH53SFFTTVNBSW9sTU5zeCsvNGhIS3h3ak5-QX4']
get_video_log_data(session_ids)

DEBUG Videolog fetching:  ['https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/2fc16dfc-7b08-458c-ba43-69e724530948/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/75a24b9d-c3e0-42a7-8d3f-40efed49e3d5/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/7ef51a22-25ee-4943-8934-b7d4355ba41f/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/8add625b-b21d-4717-8669-ab0b11ecbc96/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/ad2a9750-ded7-4a21-9ce9-934c4ff19c8e/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/b077b361-46d3-4e87-9191-4020eeafc522/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/b3a97f1e-21ba-4b45-8750-581a41e9366a/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/d2cde85d-0dad-4571-b5ff-1d5bcfc06c2b/archive.mp4', 'https://s3-ap-southeast-1.amazonaws.com/cuem

['https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/2fc16dfc-7b08-458c-ba43-69e724530948/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/75a24b9d-c3e0-42a7-8d3f-40efed49e3d5/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/7ef51a22-25ee-4943-8934-b7d4355ba41f/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/8add625b-b21d-4717-8669-ab0b11ecbc96/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/ad2a9750-ded7-4a21-9ce9-934c4ff19c8e/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/b077b361-46d3-4e87-9191-4020eeafc522/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/b3a97f1e-21ba-4b45-8750-581a41e9366a/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/46187592/d2cde85d-0dad-4571-b5ff-1d5bcfc06c2b/archive.mp4',
 'https://s3-ap-southeast-1.amazonaws.com/cuemath-tokbox/4618759

In [3]:


import ast
#print(df1)
df1["user_id"] = [ast.literal_eval(x['stream']['connection']['data'])['userId'] for idx, x in enumerate(list(df1['_activity_data']))]
df1.iloc[0]['session_id']


'2_MX40NjE4NzU5Mn5-MTU0MDQ2MjEyNTM0N35MUERTOXVCWmlUNGVjZm1LTGNiMkYySHF-QX4'

In [63]:
students = student_collection.collection.find({})
counter = 0
for idx, user in enumerate(students):
    #print(idx,user['classwork'].items())
    for activity_id, session_dict in user['homework'].items():
        #print(idx, session_dict.keys())
        act_done = False
        for session_id, session_info in session_dict.items():
            if(session_info['session_cleared'] == True):
                act_done = True
                counter+=1
        if(not act_done):
            print(idx, user['_id'],"   :   ", activity_id, "   :   ", act_done)

print(counter)

0 cd91e128-d1e9-11e8-bc88-02420a00002e    :    4160f6d4-c545-11e8-9bdf-c4b301c785cf    :    False
29 16e709fc-cbaa-11e8-be07-02420a00002d    :    2b0de39c-d76a-11e8-8cb1-c4b301c785cf    :    False
42 43121716-d5f9-11e8-a0a7-02420a00002f    :    008a3322-0369-11e9-8162-c4b301c785cf    :    False
47 8d7e1768-d1ea-11e8-b4c6-02420a000034    :    bc9dbe1c-df60-11e8-aba6-c4b301c785cf    :    False
50 fc491b36-f6e1-11e8-a967-02420a000012    :    4090ce34-c545-11e8-b61d-c4b301c785cf    :    False
86 bb0dd27c-c501-11e8-a7d2-02420a0000dd    :    dff26c94-ed6d-11e8-b6f1-c4b301c785cf    :    False
92 c81f2c4a-d762-11e8-84fc-02420a0000b2    :    3fcdd654-c545-11e8-801c-c4b301c785cf    :    False
94 56b5f4be-f6dc-11e8-b02f-02420a0000e9    :    3fcdd654-c545-11e8-801c-c4b301c785cf    :    False
99 d2c71e08-d06b-11e8-8780-02420a00002a    :    006e2c7a-0369-11e9-9d21-c4b301c785cf    :    False
105 a51a0e06-d5f8-11e8-a0a7-02420a00002f    :    2ac4fda8-d76a-11e8-ab5c-c4b301c785cf    :    False
120 0e7114

In [47]:
students = student_collection.collection.find({'_id':'bac39e14-c501-11e8-a7d2-02420a0000dd'})
counter = 0
for idx, user in enumerate(students):
    if(len(user['classwork'].keys())>0):
        max_dt = max(user['classwork'].keys())
        last_class = classroom_collection.collection.find_one({'_id':user['classwork'][max_dt]})
        print(last_class)
print(counter)

{'_id': '00149e68-18f5-11e9-96ea-02420a00016e', 'student_id': 'bac39e14-c501-11e8-a7d2-02420a0000dd', 'teacher_id': '951d5ddc-fd7e-11e5-926f-0698d46ebbc3', 'date': '2019-01-16', 'status': 'P', 'teacher_num_attendees': 7, 'teacher_status': 'P', 'scheduled_start_time': '17:30:00', 'scheduled_end_time': '18:30:00', 'sessions_attempted': {'63a491b8-1986-11e9-b70c-02420a000166': {'item_response_dict': {'b950a851-cca4-4c01-be3f-3d0454dd7d22': {'d7b0d2d2-e523-4036-9103-34aa25c360b0': '2019-01-16T12:02:36Z'}, '8b6c024e-43bf-4692-b724-54545e53abd3': {'523cb4a0-bf34-482c-abad-d75b21986bf3': '2019-01-16T12:03:31Z'}, '3b5a2eff-6f37-436a-9836-68994a9cac95': {'d8d4002c-3aa9-4358-a8fb-de20a35689b1': '2019-01-16T12:05:25Z'}, 'f90ae058-ab9e-4971-888f-9043b911e205': {'1b004ddc-faf8-45d8-a343-0bbae84240ce': '2019-01-16T12:07:47Z'}, '3183b09a-13fb-4ef3-ae76-ce971cca16ec': {'6c9ac8de-b3d5-4585-ba4d-179d1c1f8a9e': '2019-01-16T12:09:38Z'}, '84059415-910a-44f7-bdf7-f1c800285cd2': {'22861c40-c1f3-4766-8f3e-b97

In [28]:
import statistics
assessments = activity_collection.collection.find({'type':'ASSESSMENT','program_code':{'$regex':'CBSE'}})
median_score_list = {}
for activity in assessments:
    if(len(activity['distribution_time'])>0):
        median_score_list.update({(activity['reference'] +"."+ activity['program_code']):statistics.median(activity['distribution_time'].values())})
    else:
        median_score_list.update({(activity['reference'] +"."+ activity['program_code']):0})
sorted_by_value = sorted(median_score_list.items(), key=lambda kv: kv[1])
sorted_by_value



[('assessment-commercial-math.CBSE.G7', 0),
 ('assessment-commerce-recap.CBSE.G8', 0),
 ('assessment-triangles-congruence.CBSE.G7', 0),
 ('assessment-lines-angles.A01.CBSE.G7', 0),
 ('assessment-solid-shapes.CBSE.G7', 515.0),
 ('assessment-algebra.CBSE.G7', 877.0),
 ('assessment-exponents.CBSE.G7', 1289.5),
 ('assessment-coordinate-geometry-basics.CBSE.G9', 1584.5),
 ('assessment-exponents-recap.CBSE.G8', 1660.0),
 ('assessment-2v-linear-equations.CBSE.G9', 1694.0),
 ('assessment-powers-roots.CBSE.G7', 1802.0),
 ('assessment-cubes-and-roots.CBSE.G8', 1827.0),
 ('assessment-triangles.CBSE.G9', 1853.5),
 ('assessment-quadrilaterals.CBSE.G9', 1956.0),
 ('assessment-circles.CBSE.G9', 1964.5),
 ('assessment-lines-angles.CBSE.G9', 2025.0),
 ('assessment-perimeter-area-recap.CBSE.G9', 2167.0),
 ('assessment-mensuration.CBSE.G9', 2358.0),
 ('assessment-percentages.CBSE.G7', 2381.0)]

In [45]:
classes = classroom_collection.collection.find({'status':"P"})
all_classes = classroom_collection.collection.count_documents({})
data = {}
count_class = 0
for classroom in classes:
    call_tm = 0
    if(len(classroom['av_data'])>0):
        for calls in classroom['av_data']['call_data']:
            if(calls['duration_in_seconds'] > 12):
                call_tm += calls['duration_in_seconds']
    try:
        data[classroom['student_id']] += call_tm
    except KeyError:
        data[classroom['student_id']] = call_tm
statistics.mean(data.values())

4504.85937646383

In [57]:
classes = classroom_collection.collection.find({'status':"P"})
all_classes = classroom_collection.collection.count_documents({})
data = {}
count_class = 0
for classroom in classes:
    call_tm = 0
    if(len(classroom['sessions_attempted'])>0):
        try:
            data[classroom['student_id']][0] += len(classroom['sessions_attempted'])
            data[classroom['student_id']][1] += 1
        except KeyError:
            data[classroom['student_id']] = [len(classroom['sessions_attempted']), 1]
chutad = []
for k, v in data.items():
    #if((float(v[0]))<2):
    chutad.append(v[0]/v[1])
statistics.median(chutad)

2.0172413793103448

In [60]:
with open("./Analysis/ram_file_avgs.csv", "w") as fp:
    fp.write("Session_id, User_id, avg_incorrect_streak, accuracy_per_item,  \n")
for session in session_collection.collection.find({}):
    

{'_id': '09e64744-faf9-11e8-9b03-02420a0000d3', 'created_on': datetime.datetime(2018, 12, 8, 20, 23, 35, 468000), 'updated_on': datetime.datetime(2018, 12, 8, 20, 23, 35, 468000), 'email': 'grade_7_a@cuemath.com', 'phone': '9877736623', 'first_name': 'Grade7a', 'last_name': 'Student', 'gender': 'M', 'is_demo': True, 'is_teacher': False, 'sessions_attempted': []}
{'_id': '1293fe6c-ce13-11e8-b656-02420a00002b', 'created_on': datetime.datetime(2018, 10, 12, 17, 6, 34, 618000), 'updated_on': datetime.datetime(2018, 10, 16, 17, 5, 53, 530000), 'email': 'demoseven2022@cuemath.com', 'phone': None, 'first_name': 'Demo', 'last_name': 'Seven', 'gender': 'M', 'is_demo': True, 'is_teacher': False, 'sessions_attempted': []}
{'_id': '1a89b6c2-d780-11e8-a338-02420a0000b0', 'created_on': datetime.datetime(2018, 10, 24, 16, 59, 43, 581000), 'updated_on': datetime.datetime(2018, 10, 24, 16, 59, 43, 581000), 'email': 'studentz@cuemath.com', 'phone': '', 'first_name': 'Student', 'last_name': 'Z', 'gender'

In [20]:
item_median_time_list = []
fl_nm_item = "./item_score_time_list.csv"
items = item_collection.collection.find()
with open(fl_nm_item, "w") as fp:
    fp.write("Item ID, Item reference, Median Time, 75%ile time, 80%ile time, Median Score, 75%ile Score, 80%ile Score \n")
for item in items:
    item_time_dict = item.get('distribution_time', {})
    item_score_dict = item.get('distribution_score', {})
    if(len(item_time_dict)>0):
        median_time = np.percentile((np.array(list(item_time_dict.values()), dtype=np.float64)/len(item['questions'])), 50)
        percentile_75 = np.percentile((np.array(list(item_time_dict.values()), dtype=np.float64)/len(item['questions'])), 75)
        percentile_80 = np.percentile((np.array(list(item_time_dict.values()), dtype=np.float64)/len(item['questions'])), 80)    
    if(len(item_score_dict)>0):
        median_score = np.percentile((np.array(list(item_score_dict.values()), dtype=np.float64)/item['max_score']), 50)
        score_percentile_75 = np.percentile((np.array(list(item_score_dict.values()), dtype=np.float64)/item['max_score']), 75)
        score_percentile_80 = np.percentile((np.array(list(item_score_dict.values()), dtype=np.float64)/item['max_score']), 80)            
        with open(fl_nm_item, "a") as fp:
            fp.write(item['_id']+", "+item['reference'].replace(', ', "-") + ", " + str(median_time) + \
                     ", " + str(percentile_75) + ", " + str(percentile_80) + ", " + str(median_score) + ", " + \
                     str(score_percentile_75) + ", " + str(score_percentile_80) +"\n")


In [62]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


trace1 = go.Box(
  y = [2, 3, 1, 5],
  x = ["A", "A", "A", "A"],
  line = dict(color = 'gray'),
  name = "A",
  orientation = "v"
)

trace2 = go.Box(
  y = [8, 3, 6, 5],
  x = ["B", "B", "B", "B"],
  line = dict(color = 'gray'),
  name = "B",
  orientation = "v"
)

trace3 = go.Box(
  y = [2, 3, 2, 5],
  x = ["C", "C", "C", "C"],
  line = dict(color = 'gray'),
  name = "C",
  orientation = "v"
)

trace4 = go.Box(
  y = [7.5, 3, 6, 4],
  x = ["D", "D", "D", "D"],
  line = dict(color = 'gray'),
  name = "D",
  orientation = "v"
)

data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
  annotations = [
    dict(
      y = -0.0951769406393,
      x = 1.06972670892,
      showarrow = False,
      text = "Subgroup",
      xref = "paper",
      yref = "paper"
    ),
    dict(
      y = -0.235516552511,
      x = 1.07060587474,
      showarrow = False,
      text = "Group",
      xref = "paper",
      yref = "paper"
    ),
    dict(
      y = -0.235516552511,
      x = 0.922906017856,
      showarrow = False,
      text = "One",
      xref = "paper",
      yref = "paper"
    ),
    dict(
      y = -0.235516552511,
      x = 0.375,
      showarrow = False,
      text = "Two",
      xref = "paper",
      yref = "paper"
    )
  ],
  height = 400,
  hovermode = "closest",
  legend = dict(
    y = 0.986145833333,
    x = 0.936263886049
  ),
  margin = dict(
    r = 10,
    t = 25,
    b = 40,
    l = 110
  ),
  shapes = [
    dict(
      line = dict(
        color = "rgba(68, 68, 68, 0.5)",
        width = 1
      ),
      type = "line",
      y0 = -0.3,
      y1 = 1.2,
      xref = "paper",
      x0 = 0.5,
      x1 = 0.5,
      yref = "paper"
    ),
    dict(
      line = dict(
        color = "rgba(68, 68, 68, 0.63)",
        width = 1
      ),
      type = "line",
      y0 = -0.3,
      y1 = 1.2,
      xref = "paper",
      x0 = 1,
      x1 = 1,
      yref = "paper"
    )
  ],
  showlegend = True,
  title = "",
  width = 600,
  yaxis = dict(
    domain = [0, 1]
  ),
  xaxis = dict(
    autorange = True,
    categoryorder = "category descending",
    domain = [0, 1],
    range = [-0.5, 3.5],
    showline = True,
    title = "",
    type = "category"
  )
)

fig = go.Figure(data=data, layout=layout)
plot(fig, filename = 'subcategory-axes')

'file:///Users/nadeeshgarg/Documents/Learnosity_Data_Extraction/scripts/subcategory-axes.html'